<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Download_NDVI_Image_Collection_Netherlands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Authenticate and Initialise
import ee

ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=szZBdIVlkNQkwnLgKeqPrWgjIqp5qljfAMgz12-AzqY&tc=4h0-1yvyVBlQgEPTok-2wBJxGQZ-4Q3ZnLHy2Om8cXY&cc=KStgTQ1MtQNPEn4z9htpCddXnefFJ6uAV5SZDFW1nWw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnL8tHjV4-gbqYPPLonnzEvWIDCL9n82yNzX_VEdwfNU-l8qXVhnfU

Successfully saved authorization token.


In [19]:
# Collection creation

#Netherlands Geometry
geometry = ee.Geometry.Point([5.1214, 52.3676])

#location = roi.geometry().centroid().coordinates().getInfo()[::-1]
location = geometry.coordinates().getInfo()[::-1]

#Unique ee ID
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

# Write a function for Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

filtered = s2.filter(ee.Filter.date('2020-01-01', '2020-05-01')) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds)

# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)
median = withNdvi.median()

In [28]:
#Displaying Maps

# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def addLayer (self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.addLayer = addLayer

# Create the map object.
Map = folium.Map(location=location, zoom_start = 8)

In [29]:
Map.addLayer(median, rgbVis, 'Sentinel')

#Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
display(Map)

In [30]:
#Export all Images Exports
image_ids = withNdvi.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  18


In [33]:
# Export with 100m resolution for this demo
for i, image_id in enumerate(image_ids):
  image = ee.Image(withNdvi.filter(ee.Filter.eq('system:index', image_id)).first())
  task = ee.batch.Export.image.toDrive(**{
    'image': image.select('ndvi'),
    'description': 'Image Export {}'.format(i+1),
    'fileNamePrefix': image.id().getInfo(),
    'folder':'Lecture_8',
    'scale': 100,
    'region': image.geometry().bounds().getInfo()['coordinates'],
    'maxPixels': 1e10
  })
  task.start()
  print('Started Task: ', i+1)

Started Task:  1
Started Task:  2
Started Task:  3
Started Task:  4
Started Task:  5
Started Task:  6
Started Task:  7
Started Task:  8
Started Task:  9
Started Task:  10
Started Task:  11
Started Task:  12
Started Task:  13
Started Task:  14
Started Task:  15
Started Task:  16
Started Task:  17
Started Task:  18
